In [1]:
from hamiltonian import Hamiltonian, HamiltonianH2, HamiltonianW

In [2]:
lih = Hamiltonian('LiH', 1.5)
beh2 = Hamiltonian('BeH2', 1.3)

h2_jw = HamiltonianH2('jw')
h2_parity = HamiltonianH2('parity')
h2_bk = HamiltonianH2('bk')

water_jw = HamiltonianW('jw')
water_parity = HamiltonianW('parity')
water_bk = HamiltonianW('bk')

In [3]:
%time lih.pauli_rep.ground()
%time beh2.pauli_rep.ground()
%time h2_jw.pauli_rep.ground()

%time energy, state = water_jw.pauli_rep.ground()

CPU times: user 106 ms, sys: 2.75 ms, total: 109 ms
Wall time: 107 ms
CPU times: user 242 ms, sys: 564 µs, total: 243 ms
Wall time: 243 ms
CPU times: user 524 ms, sys: 32.4 ms, total: 556 ms
Wall time: 398 ms
CPU times: user 22.6 s, sys: 2.37 s, total: 25 s
Wall time: 9.63 s


In [4]:
from sparse import energy as sparseenergy

pr = beh2.pauli_rep
energy, state = pr.ground()
print("energy: ", energy)
energy_sparse = sparseenergy(pr, state)
print("energy_sparse: ", energy_sparse)

energy:  -3.9893688805012433
energy_sparse:  -3.989368880501246


In [5]:
β = pr.local_dists_uniform()
%time pr.variance_local(energy, state, β)

CPU times: user 10.9 s, sys: 20.7 ms, total: 10.9 s
Wall time: 10.9 s


18.179850794404423

In [6]:
print(pr.variance_ell_1(energy))

β = pr.local_dists_pnorm(1)
print(pr.variance_local(energy, state, β))

β = pr.local_dists_pnorm(2)
print(pr.variance_local(energy, state, β))

β = pr.local_dists_pnorm('infinity')
print(pr.variance_local(energy, state, β))

42.28949301890357
88.57935992546825
80.69169432668937
79.62377675669387


# Variance optimisation

If we use a uniform/biased pauli/magic bases algorithm to estimate the energy, and we assume that the state is a product-state (or close to a produce-state) then we end up wanting to minimise:
$$
    \Var[\nu]
    =
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    +
    \sum_{\Qarrow,\Rarrow \in \{I,Z\}^{\otimes n}} \alpha_\Qarrow \alpha_\Rarrow \prod_{i} m_i(1-\delta_{Q_i R_i})
    - \tr(H_0\rho)^2
$$
(Above is the chemistry HF situation, where the product-state is in the $Z$-basis, but this statement wouldn't depend on using $Z$-basis uniformly.)

We end up wanting to minimise:
$$
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    \qquad
    \textrm{subject to}
    \qquad
    \beta_{i,X}+\beta_{i,Y}+\beta_{i,Z}=1 \,\forall i,
    \qquad
    \beta_{i,P}\ge 0
$$

In [7]:
β_optimal = pr.local_dists_optimal()
print(pr.variance_local(energy, state, β_optimal))

13.84508849193289


In [8]:
def variances_dict(pauli_rep):
    pr = pauli_rep
    dic = {}
    
    energy, state = pr.ground()
    print("energy :", energy)

    # ell_1
    var = pr.variance_ell_1(energy)
    print("ell 1: ", var)
    dic['ell_1'] = var
    
    # uniform
    β = pr.local_dists_uniform()
    var = pr.variance_local(energy, state, β)
    print("uniform: ", var)
    dic['uniform'] = var
    
    # optimal
    β = pr.local_dists_optimal()
    var = pr.variance_local(energy, state, β)
    print("optimal: ", var)
    dic['optimal'] = var
    
    return dic

In [9]:
variances_ALL = {}
beta_optimal_ALL = {}

In [10]:
name = 'lih'
pr = lih.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.101406845059899
ell 1:  8.798891105500468
uniform:  2.2307301610389207
optimal:  1.576299391494913
CPU times: user 4.86 s, sys: 83.3 ms, total: 4.95 s
Wall time: 4.33 s


In [11]:
name = 'beh2'
pr = beh2.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -3.989368880501244
ell 1:  42.28949301890357
uniform:  18.179850794404405
optimal:  13.8450884919329
CPU times: user 25.6 s, sys: 430 ms, total: 26 s
Wall time: 22.6 s


In [12]:
name = 'h2_jw'
pr = h2_jw.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.860860555520753
ell 1:  119.67906001905914
uniform:  51.39982021387585
optimal:  17.742071936811314
CPU times: user 43.9 s, sys: 1.01 s, total: 44.9 s
Wall time: 36.8 s


In [13]:
name = 'h2_parity'
pr = h2_parity.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

/opt/anaconda3/envs/qiskit18/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


energy : -1.8608605555207602
ell 1:  119.67906001905916
uniform:  70.76147441962266
optimal:  18.924750112035003
CPU times: user 1min 9s, sys: 3.74 s, total: 1min 13s
Wall time: 43.5 s


In [14]:
name = 'h2_bk'
pr = h2_bk.pauli_rep

beta_optimal_ALL[name] = pr.local_dists_optimal()
%time variances_ALL[name] = variances_dict(pr)

energy : -1.8608605555207667
ell 1:  119.67906001905938
uniform:  168.86600324405464
optimal:  19.516932420931518
CPU times: user 54.2 s, sys: 2.59 s, total: 56.8 s
Wall time: 38.9 s


# Water with 14 qubits takes a *long* time

In [15]:
pr = water_jw.pauli_rep
%time β = pr.local_dists_optimal()
β

CPU times: user 6min 53s, sys: 1min 1s, total: 7min 54s
Wall time: 59.1 s


{0: [0.43257858556565915, 0.43257858722135467, 0.1348428272129862],
 1: [0.24098007537433425, 0.2409800708577077, 0.518039853767958],
 2: [0.13870541903721406, 0.13870542588015417, 0.7225891550826317],
 3: [0.13957872152763764, 0.13957871134125624, 0.7208425671311062],
 4: [0.16984472160573807, 0.1698447243114196, 0.6603105540828423],
 5: [0.2675208449775755, 0.2675208434379076, 0.464958311584517],
 6: [0.11647067744295292, 0.11647067819430176, 0.7670586443627454],
 7: [0.43257857708372555, 0.43257860312684154, 0.13484281978943294],
 8: [0.24098005034367995, 0.24098008250406633, 0.5180398671522537],
 9: [0.13870542248842968, 0.13870542724539398, 0.7225891502661764],
 10: [0.13957872657623138, 0.13957872195005588, 0.7208425514737127],
 11: [0.16984475333796817, 0.16984473040786618, 0.6603105162541657],
 12: [0.2675208758292949, 0.26752084565650264, 0.46495827851420257],
 13: [0.11647066971784335, 0.1164706702788042, 0.7670586600033525]}

In [16]:
%time energy, state = pr.ground()

CPU times: user 24.8 s, sys: 2.67 s, total: 27.5 s
Wall time: 10.1 s


In [17]:
%time var = pr.variance_ell_1(energy)
print("ell 1: ", var)

CPU times: user 75 µs, sys: 11 µs, total: 86 µs
Wall time: 12.9 µs
ell 1:  4363.497773126064


In [ ]:
%time var = pr.variance_local(energy, state, pr.local_dists_uniform())
print("uniform: ", var)

In [ ]:
%time var = pr.variance_local(energy, state, β)
print("optimal: ", var)